In [ ]:
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x:'%.4f'%x)
pd.set_option('display.max_rows', 300)
 
df = pd.read_csv('final_version.csv')
df

In [ ]:
## only gross
gross = df[df['commodity_transaction'] == 'Electricity - Gross production'].sort_values(['country_or_area', 'year'])
gross


In [ ]:
# only renewalbe or no-renewable
renewable_list = ['Electricity - total nuclear production',
                'Electricity - total geothermal production',
                'Electricity - total tide, wave and hydro production',
                'Electricity - total solar production',
                'Electricity - total wind production']

non_renewable_list = ['Electricity - total thermal production',
                    'From chemical heat – Autoproducer',
                    'From chemical heat – Main activity',
                    'From other sources – Autoproducer',
                    'From other sources – Main activity']

rene = df[(df['commodity_transaction'].isin(renewable_list))|(df['commodity_transaction'].isin(non_renewable_list))].sort_values(['country_or_area', 'year'])
rene

In [ ]:
def f_renewable(row):
    if row in renewable_list:
        return 1
    else:
        return 0

def f_non_renewable(row):
    if row in non_renewable_list:
        return 1
    else:
        return 0



In [ ]:
rene['renewable'] = rene['commodity_transaction'].apply(f_renewable)
rene['non_renewable'] = rene['commodity_transaction'].apply(f_non_renewable)


In [ ]:

rene['renewable'] = rene['new_quantity'] * rene['renewable']
rene['non_renewable'] = rene['new_quantity'] * rene['non_renewable']
rene

In [ ]:
# rene.drop(rene[(rene['commodity_transaction'] == 'From other sources – Main activity') & (rene['category']=='heat')].index, inplace = True)
# rene.drop(rene[(rene['commodity_transaction'] == 'From other sources – Autoproducer') & (rene['category']=='heat')].index, inplace = True)

In [ ]:
x = rene.groupby(['country_or_area', 'ISO', 'year']).agg({'renewable':'sum', 'non_renewable': 'sum'}).reset_index()
x

In [ ]:
y = pd.merge(gross, x, on = ['country_or_area','ISO','year'], how= 'inner')
y

In [ ]:
#checking 
y['difference'] =round( y['new_quantity'] - y['renewable'] - y['non_renewable'],4)
y

In [ ]:
y[y['difference']!=0]

In [ ]:
len(y[y['difference']!=0])

In [ ]:
## add procent of renewable i non renewable
y['%_renewable'] = (y['renewable']/ y['new_quantity']) *100
y['%_non_renewable'] = (y['non_renewable']/ y['new_quantity']) *100
# y.insert(loc=7, column = 'All_consumption', value =  (y['new_quantity']/1000000))
# y.insert(loc=8, column = 'Renewable', value =  (y['renewable']/1000000))
# y.insert(loc=9, column = 'Non_renewable', value =  (y['non_renewable']/1000000))
# y['new_unit'] = 'TWh'
# y['new_quantity'] = y['new_quantity']/1000000
# y['renewable'] = y['renewable']/1000000
# y['non_renewable'] = y['non_renewable']/1000000
y

In [ ]:
y.to_csv('renewable.csv', index = False)